In [30]:
##Normal AR
import numpy as np
import pickle
import math
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            pi_action = get_action(model,obs,True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################
maxx={"seed":None,"R100":0,"numep":0}
#sed=[30,69,75,100,343,2048,5000,6557,7731,8888,9351]
#best=30
sed=[75,69,30,100,2048,5000,6557,7731,8888,9351]
#sed=[30,69,75,100,343]#best onLLC
#sed=[100]
summ100ret=0
summret=0
def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]
for seed in sed:
    best=343
    import gym
    import random
    import numpy as np
    import os
    #import matplotlib.pyplot as plt
    import torch
    from datetime import datetime, timedelta
    import time
    from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
    from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
    #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
    #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 
    #name="LunarLanderContinuous-v2"
    name="Walker2d-v2"
    tp="ou_noise/nr_mdp_0.1_10"
    seeding(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    test_env = gym.make(name)
    obs_dim = test_env.observation_space.shape[0]
    act_dim = test_env.action_space.shape[0]
    act_limit = test_env.action_space.high[0]
    
    encoder_pop_dim=10
    decoder_pop_dim=10
    
    neuron_input_memory = []
    input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
    # Set core list in a special format easier for reset layer by layer
    if name != "Ant-v3":
        encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
        bias_core_list = [1 for _ in range(3)]
        decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
        hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
    else:
        encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
        bias_core_list = [5 for _ in range(3)]
        decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
        hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                  for layer in range(2)]
    core_list = [encoder_core_list]
    core_list.extend(hidden_layer_core_list)
    core_list.extend([decoder_core_list, bias_core_list])
    

    device = torch.device("cpu")
    m=0#0th model
    result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/6quantile_doublerobust/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
    test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
    best_epoch_idx = 0
    best_epoch_reward = 0
    for idx in range(20):
        if test_reward[(idx + 1) * 5 - 1] > best_epoch_reward:
            best_epoch_reward = test_reward[(idx + 1) * 5 - 1]
            best_epoch_idx = (idx + 1) * 5
    print("#######use epoch: ",best_epoch_idx)        
    model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
    #buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
    encoder_var=0.15
    std=math.sqrt(encoder_var)
    spike_ts=5
    
    # Replay buffer for running z-score norm
    #b_mean_var = pickle.load(open(buffer_dir, "rb"))
    #replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
      #                           clip_limit=3, norm_update_every=1)
    #replay_buffer.mean = b_mean_var[0]
    #replay_buffer.var = b_mean_var[1]
    

    popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                           decoder_pop_dim, (256, 256),
                               (-3, 3), std, spike_ts, act_limit, device)
    popsan.load_state_dict(torch.load(model_dir))
    
    hidden_size=256
    #env = NormalizedActions(env)
    test_env.seed(seed)
    test_env.action_space.seed(seed)
    test_env.observation_space.seed(seed)
    #model.eval()
    print("#"*50)

    mean_ep_reward=evaluate_policy(popsan, test_env, num_steps=1000, deterministic=True,seed=seed)
    print("seed",seed,"Mean_R",mean_ep_reward)
    if mean_ep_reward>maxx["R100"]:
        maxx["R100"]=mean_ep_reward
        maxx["seed"]=seed
    summret=summret+mean_ep_reward
print("#"*100,"\n",maxx,"\n","#"*100)
################Print AVG on other 10 seeds
print("sum",summret,"AVG:",summret/len(sed))

#######use epoch:  90
[17, 256, 256, 6]
##################################################
ep: 0  i 14 2.301454712931467
ep: 1  i 14 0.9387336080972943
ep: 2  i 14 0.6597109381048021
ep: 3  i 18 3.448824103667464
ep: 4  i 16 1.6356813278015956
seed 75 Mean_R 1.8
#######use epoch:  90
[17, 256, 256, 6]
##################################################
ep: 0  i 15 1.454229555174597
ep: 1  i 17 2.98222318289
ep: 2  i 23 9.058253475266206
ep: 3  i 16 1.836788669069239
ep: 4  i 16 0.5865481690656715
seed 69 Mean_R 3.2
#######use epoch:  90
[17, 256, 256, 6]
##################################################
ep: 0  i 17 2.2357684783443
ep: 1  i 14 0.787380637519702
ep: 2  i 16 3.4918058322574206
ep: 3  i 16 4.160189638832572
ep: 4  i 16 6.909170929879117
seed 30 Mean_R 3.5
#######use epoch:  90
[17, 256, 256, 6]
##################################################
ep: 0  i 16 3.533702207020104
ep: 1  i 15 1.8141443105858324
ep: 2  i 14 2.274893113168351
ep: 3  i 14 1.7733599825281223
ep: 4  i

In [1]:
##Normal AR
import numpy as np
import pickle
import math
import sys
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,replay_buffer,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            pi_action = get_action(model,replay_buffer.normalize_obs(obs),True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################
maxx={"seed":None,"R100":0,"numep":0}
#sed=[30,69,75,100,343,2048,5000,6557,7731,8888,9351]
#best=30
sed=[100]#0,100,2048,5000,6557,7731,8888,9351]
#sed=[30,69,75,100,343]#best onLLC
#sed=[100]
summ100ret=0
summret=0
def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]
for seed in sed:
    best=343
    import gym
    import random
    import numpy as np
    import os
    #import matplotlib.pyplot as plt
    import torch
    from datetime import datetime, timedelta
    import time
    from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
    from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
    #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
    #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 
    #name="LunarLanderContinuous-v2"
    name="Walker2d-v2"
    tp="ou_noise/nr_mdp_0.1_10"
    seeding(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    test_env = gym.make(name)
    obs_dim = test_env.observation_space.shape[0]
    act_dim = test_env.action_space.shape[0]
    act_limit = test_env.action_space.high[0]
    
    encoder_pop_dim=10
    decoder_pop_dim=10
    
    neuron_input_memory = []
    input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
    # Set core list in a special format easier for reset layer by layer
    if name != "Ant-v3":
        encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
        bias_core_list = [1 for _ in range(3)]
        decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
        hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
    else:
        encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
        bias_core_list = [5 for _ in range(3)]
        decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
        hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                  for layer in range(2)]
    core_list = [encoder_core_list]
    core_list.extend(hidden_layer_core_list)
    core_list.extend([decoder_core_list, bias_core_list])
    

    device = torch.device("cpu")
    m=0#0th model
    #fold="5improved_result_on_quantile"
    fold="4"
    result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
    test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
    best_epoch_idx = 0
    best_epoch_reward = 0
    for idx in range(20):
        if test_reward[(idx + 1) * 5 - 1] > best_epoch_reward:
            best_epoch_reward = test_reward[(idx + 1) * 5 - 1]
            best_epoch_idx = (idx + 1)* 5
    #best_epoch_idx=100
    print("#######use epoch: ",best_epoch_idx)        
    model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
    buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
    encoder_var=0.15
    std=math.sqrt(encoder_var)
    spike_ts=5
    
    # Replay buffer for running z-score norm
    b_mean_var = pickle.load(open(buffer_dir, "rb"))
    replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
                                clip_limit=3, norm_update_every=1)
    replay_buffer.mean =b_mean_var[0]
    replay_buffer.var = b_mean_var[1]
    
    print("mean, var",b_mean_var[0],b_mean_var[1])
    popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                           decoder_pop_dim, (256, 256),
                               (-3, 3), std, spike_ts, act_limit, device)
    popsan.load_state_dict(torch.load(model_dir))
    
    #env = NormalizedActions(env)
    test_env.seed(seed)
    test_env.action_space.seed(seed)
    test_env.observation_space.seed(seed)
    #model.eval()
    print("#"*50)

    mean_ep_reward=evaluate_policy(popsan, replay_buffer, test_env, num_steps=1000, deterministic=True,seed=seed)
    print("seed",seed,"Mean_R",mean_ep_reward)
    if mean_ep_reward>maxx["R100"]:
        maxx["R100"]=mean_ep_reward
        
        maxx["seed"]=seed
    summret=summret+mean_ep_reward
print("#"*100,"\n",maxx,"\n","#"*100)
################Print AVG on other 10 seeds
print("sum",summret,"AVG:",summret/len(sed))

#######use epoch:  100
mean, var [ 1.1109331  -0.48841116 -0.05246255 -1.5781772   0.56921935 -1.1020356
 -0.1642681   0.09514029  2.836563   -0.06505611 -0.12013905 -0.09653213
 -0.1891899   0.05683086 -0.23974058 -0.17263895  0.55165493] [6.2820697e-03 9.6627899e-02 4.2779796e-02 6.2439138e-01 1.9632985e-01
 3.4452766e-01 1.5717688e-01 4.0177304e-01 1.9841712e+00 5.0324476e-01
 1.1210849e+01 7.1578236e+00 4.2817772e+01 2.3022852e+01 2.2721893e+01
 1.3312014e+01 4.2253716e+01]
[17, 256, 256, 6]
##################################################
ep: 0  i 1000 4786.478663144332
ep: 1  i 1000 4905.329915483419
ep: 2  i 718 3443.5334237632583
ep: 3  i 554 2464.370869946088
ep: 4  i 666 3115.9664351315987
seed 100 Mean_R 3743.1
#################################################################################################### 
 {'seed': 100, 'R100': 3743.1, 'numep': 0} 
 ####################################################################################################
sum 3743.1 AVG: 37

6quantile_doublerobust_rbdata: 4791
55improved_result_on_quantile: 5232
4org:  4838

In [69]:
ur="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/6quantile_doublerobust_rbdata/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10/data0_train.p"
with open(ur, 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [70]:
content

[999999,
 array([[-0.94883645,  0.03607666, -0.9098367 ,  0.00412775,  0.86604726,
          0.9923495 ]], dtype=float32),
 array([  1.02281534,   0.25103698,  -0.72569268,   0.02058063,
          0.14963572,  -0.66874077,   0.02995621,   0.82190696,
          1.77567118,  -1.07631051,   2.01581096,   0.32017107,
          0.14455479, -10.        ,   0.44586495,  -0.52327223,
         -0.01187389]),
 3.027612000833295,
 False,
 {}]

##### Important factor for test results---->
b_mean_var:--- mean,var

In [12]:
##Normal AR
import numpy as np
import pickle
import math
import sys
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,replay_buffer,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None,d=None,noise=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            obs[d]=obs[d]+np.random.normal(0,noise)
            pi_action = get_action(model,replay_buffer.normalize_obs(obs),True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                #print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################

def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]

for noise in [1,0.8,0.6,0.4,0.2]:
    print("\n\n","#"*20,"Noise=",noise,"#"*20,"\n")
    mer=[]
    for dim in range(17):
        print("\n","@"*10,dim,"@"*10,"\n")
        
        maxx={"seed":None,"R100":0,"numep":0}
        sed=[343,75,69,30,2048,5000,6557,7731,8888,9351]
        summ100ret=0
        summret=0
        for seed in sed:
     
            import gym
            import random
            import numpy as np
            import os
            #import matplotlib.pyplot as plt
            import torch
            from datetime import datetime, timedelta
            import time
            from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
            #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 
            #name="LunarLanderContinuous-v2"
            name="Walker2d-v2"
            seeding(seed)
            os.environ['PYTHONHASHSEED']=str(seed)
            test_env = gym.make(name)
            obs_dim = test_env.observation_space.shape[0]
            act_dim = test_env.action_space.shape[0]
            act_limit = test_env.action_space.high[0]

            encoder_pop_dim=10
            decoder_pop_dim=10

            neuron_input_memory = []
            input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
            # Set core list in a special format easier for reset layer by layer
            if name != "Ant-v3":
                encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [1 for _ in range(3)]
                decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
            else:
                encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [5 for _ in range(3)]
                decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                          for layer in range(2)]
            core_list = [encoder_core_list]
            core_list.extend(hidden_layer_core_list)
            core_list.extend([decoder_core_list, bias_core_list])


            device = torch.device("cpu")
            m=0#0th model
            #fold="5improved_result_on_quantile"
            #fold="6quantile_doublerobust_rbdata"
            #fold="11_snn_quantile_selu_gausactor"
            #fold="12_snn_quan_selu_gau_drop"
            #fold="14_snn_quan_adaptive_alpha"
            
            #result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"

            result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
            test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
            best_epoch_idx = 0
            best_epoch_reward = 0
            for idx in range(20):
                if test_reward[(idx + 1) * 5 - 1] > best_epoch_reward:
                    best_epoch_reward = test_reward[(idx + 1) * 5 - 1]
                    best_epoch_idx = (idx + 1)* 5
            #best_epoch_idx=100
            print("#######use epoch: ",best_epoch_idx)        
            model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
            buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
            encoder_var=0.15
            std=math.sqrt(encoder_var)
            spike_ts=5

            # Replay buffer for running z-score norm
            b_mean_var = pickle.load(open(buffer_dir, "rb"))
            replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
                                        clip_limit=3, norm_update_every=1)
            replay_buffer.mean =b_mean_var[0]
            replay_buffer.var = b_mean_var[1]

            #print("mean, var",b_mean_var[0],b_mean_var[1])
            popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                                   decoder_pop_dim, (256, 256),
                                       (-3, 3), std, spike_ts, act_limit, device)

            #popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
            #                                       decoder_pop_dim, (256, 256),
             #                          (-3, 3), std, spike_ts, act_limit, device,train=False)
            popsan.load_state_dict(torch.load(model_dir))

            #env = NormalizedActions(env)
            test_env.seed(seed)
            test_env.action_space.seed(seed)
            test_env.observation_space.seed(seed)
            #model.eval()
            print("#"*50)

            mean_ep_reward=evaluate_policy(popsan, replay_buffer, test_env, num_steps=1000, deterministic=True,seed=seed,d=dim,noise=noise)
            print("seed",seed,"Mean_R",mean_ep_reward)
            if mean_ep_reward>maxx["R100"]:
                maxx["R100"]=mean_ep_reward

                maxx["seed"]=seed
            summret=summret+mean_ep_reward
        print("#"*100,"\n",maxx,"\n","#"*100)
        ################Print AVG on other 10 seeds
        print("sum",summret,"AVG:",summret/len(sed))
        mer.append(summret/len(sed))
    #path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/"
    path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/"+fold+"/"

    k="\n".join(str(item) for item in mer)
    filee=open(path+"/test_result_"+fold+"otherseed_epbest95.csv","a")
    filee.write(str(k)+"\n")
    filee.close()



 #################### Noise= 1 #################### 


 @@@@@@@@@@ 0 @@@@@@@@@@ 

[183.07256244 141.87743548 198.92977775]
[454.99310242 444.51894272 668.45708391]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [19]:
##Normal AR
import numpy as np
import pickle
import math
import sys
sys.path.append("/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl")
def evaluate_policy(model,replay_buffer,env, num_steps=1000,n_eval_episodes=10, deterministic=True,
                    render=False, callback=None, reward_threshold=None,
                    return_episode_rewards=False,seed=None,d=None,noise=None):

    episode_rewards = []
    obs =  env.reset()
    for ep in range(5):
        i=0
        ret=0
        while True:
            obs[d]=obs[d]+np.random.normal(0,noise)
            pi_action = get_action(model,replay_buffer.normalize_obs(obs),True)
            action=pi_action
            obs, rewards, dones, info = env.step(action)
            #obs=agent.Tensor([obs])
            ret += rewards
            i+=1
            if dones:
                obs = env.reset()
                #print("ep:",ep," i",i,ret)
                episode_rewards.append(ret)
                break
    mean_ep_reward=round(np.mean(episode_rewards),1)
    return mean_ep_reward

##########################Evaluate SAC###############################

def seeding(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

def get_action(ac,o, deterministic=False):
    a,_=ac(torch.as_tensor(o, dtype=torch.float32, device=device), 1,
                  deterministic) 
    a=a.to('cpu')
    k=a.detach().numpy()
    #print(k[0])
    return k[0]

for noise in [1,0.8,0.6,0.4,0.2]:
    print("\n\n","#"*20,"Noise=",noise,"#"*20,"\n")
    mer=[]
    for dim in range(17):
        print("\n","@"*10,dim,"@"*10,"\n")
        
        maxx={"seed":None,"R100":0,"numep":0}
        sed=[343,75,69,30,2048,5000,6557,7731,8888,9351]
        summ100ret=0
        summret=0
        for seed in sed:
     
            import gym
            import random
            import numpy as np
            import os
            #import matplotlib.pyplot as plt
            import torch
            from datetime import datetime, timedelta
            import time
            from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor
            #from popsan_drl.popsan_sac.replay_buffer_norm import ReplayBuffer
            #from popsan_drl.popsan_sac.popsan import SquashedGaussianPopSpikeActor 
            #name="LunarLanderContinuous-v2"
            name="HalfCheetah-v2"
            seeding(seed)
            os.environ['PYTHONHASHSEED']=str(seed)
            test_env = gym.make(name)
            obs_dim = test_env.observation_space.shape[0]
            act_dim = test_env.action_space.shape[0]
            act_limit = test_env.action_space.high[0]

            encoder_pop_dim=10
            decoder_pop_dim=10

            neuron_input_memory = []
            input_channel_data_shape = (obs_dim * 10) // 4 + (obs_dim * 10) % 4
            # Set core list in a special format easier for reset layer by layer
            if name != "Ant-v3":
                encoder_core_list = [0 for _ in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [1 for _ in range(3)]
                decoder_core_list = [2 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 128 + 3 + layer * 2 for num in range(256)] for layer in range(2)]
            else:
                encoder_core_list = [num // 222 for num in range(obs_dim * encoder_pop_dim)]
                bias_core_list = [5 for _ in range(3)]
                decoder_core_list = [6 for _ in range(act_dim * decoder_pop_dim)]
                hidden_layer_core_list = [[num // 32 + 7 + layer * 8 for num in range(256)] 
                                          for layer in range(2)]
            core_list = [encoder_core_list]
            core_list.extend(hidden_layer_core_list)
            core_list.extend([decoder_core_list, bias_core_list])


            device = torch.device("cpu")
            m=0#0th model
            #fold="5improved_result_on_quantile"
            #fold="6quantile_doublerobust_rbdata"
            #fold="11_snn_quantile_selu_gausactor"
            #fold="12_snn_quan_selu_gau_drop"
            #fold="14_snn_quan_adaptive_alpha"
            #fold="15_snn_selu_quan_many_actor"
            fold="15"
            #result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/params/spike-sac_sac-popsan-Walker2d-v2-encoder-dim-10-decoder-dim-10"
            fold_env="result_half/"
            result_dir ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/"\
                +fold_env+result+fold+"/params/spike-sac_sac-popsan-"+name+"-encoder-dim-10-decoder-dim-10"
            test_reward, _ = pickle.load(open(result_dir + '/model' + str(m) + '_test_rewards.p', 'rb'))
            best_epoch_idx = 0
            best_epoch_reward = 0
            for idx in range(20):
                #print(test_reward[0],"nn",test_reward[(idx + 1) * 5 - 1])
                mx_ac=np.argmax(test_reward[(idx + 1) * 5 - 1])
                if test_reward[(idx + 1) * 5 - 1][mx_ac] > best_epoch_reward:
                   
                    best_epoch_reward = test_reward[(idx + 1) * 5 - 1][mx_ac] 
                    best_epoch_idx = (idx + 1)* 5
                #print(best_epoch_reward)
            #best_epoch_idx=100
            print("#######use epoch: ",best_epoch_idx)        
            model_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '.pt'
            buffer_dir = result_dir + '/model' + str(m) + '_e' + str(best_epoch_idx) + '_mean_var.p'
            encoder_var=0.15
            std=math.sqrt(encoder_var)
            spike_ts=5

            # Replay buffer for running z-score norm
            b_mean_var = pickle.load(open(buffer_dir, "rb"))
            replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=1,
                                        clip_limit=3, norm_update_every=1)
            replay_buffer.mean =b_mean_var[0]
            replay_buffer.var = b_mean_var[1]

            #print("mean, var",b_mean_var[0],b_mean_var[1])
            popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
                                                   decoder_pop_dim, (256, 256),
                                       (-3, 3), std, spike_ts, act_limit, device)

            #popsan = SquashedGaussianPopSpikeActor(obs_dim, act_dim, encoder_pop_dim, 
            #                                       decoder_pop_dim, (256, 256),
             #                          (-3, 3), std, spike_ts, act_limit, device,train=False)
            popsan.load_state_dict(torch.load(model_dir))

            #env = NormalizedActions(env)
            test_env.seed(seed)
            test_env.action_space.seed(seed)
            test_env.observation_space.seed(seed)
            #model.eval()
            print("#"*50)

            mean_ep_reward=evaluate_policy(popsan, replay_buffer, test_env, num_steps=1000, deterministic=True,seed=seed,d=dim,noise=noise)
            print("seed",seed,"Mean_R",mean_ep_reward)
            if mean_ep_reward>maxx["R100"]:
                maxx["R100"]=mean_ep_reward

                maxx["seed"]=seed
            summret=summret+mean_ep_reward
        print("#"*100,"\n",maxx,"\n","#"*100)
        ################Print AVG on other 10 seeds
        print("sum",summret,"AVG:",summret/len(sed))
        mer.append(summret/len(sed))
    #path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/result/usable/"+fold+"/"
    path ="/home/admin12/Downloads/github/pc_snn/pop-spiking-deep-rl/popsan_drl/popsan_sac/"fold_env+fold+"/"

    k="\n".join(str(item) for item in mer)
    filee=open(path+"/test_result_"+fold+"otherseed_"str(best_epoch_idx)+".csv","a")
    filee.write(str(k)+"\n")
    filee.close()



 #################### Noise= 1 #################### 


 @@@@@@@@@@ 0 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): SELU()
  (4): Linear(in_features=256, out_features=6, bias=True)
)
##################################################
seed 343 Mean_R 516.8
#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncode

seed 9351 Mean_R 1234.6
#################################################################################################### 
 {'seed': 75, 'R100': 1545.4, 'numep': 0} 
 ####################################################################################################
sum 9732.4 AVG: 973.24

 @@@@@@@@@@ 1 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256, bi

seed 9351 Mean_R 44.6
#################################################################################################### 
 {'seed': 2048, 'R100': 191.1, 'numep': 0} 
 ####################################################################################################
sum 908.8000000000001 AVG: 90.88000000000001

 @@@@@@@@@@ 2 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256,

seed 9351 Mean_R 873.7
#################################################################################################### 
 {'seed': 9351, 'R100': 873.7, 'numep': 0} 
 ####################################################################################################
sum 4680.999999999999 AVG: 468.0999999999999

 @@@@@@@@@@ 3 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256

seed 9351 Mean_R 5158.2
#################################################################################################### 
 {'seed': 343, 'R100': 5182.8, 'numep': 0} 
 ####################################################################################################
sum 49562.4 AVG: 4956.24

 @@@@@@@@@@ 4 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256,

...

seed 9351 Mean_R 5477.5
#################################################################################################### 
 {'seed': 5000, 'R100': 5519.3, 'numep': 0} 
 ####################################################################################################
sum 54923.2 AVG: 5492.32

 @@@@@@@@@@ 13 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=25

seed 9351 Mean_R 5441.3
#################################################################################################### 
 {'seed': 69, 'R100': 5521.8, 'numep': 0} 
 ####################################################################################################
sum 54814.6 AVG: 5481.46

 @@@@@@@@@@ 14 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=256,

seed 9351 Mean_R 5465.2
#################################################################################################### 
 {'seed': 2048, 'R100': 5511.6, 'numep': 0} 
 ####################################################################################################
sum 54798.7 AVG: 5479.87

 @@@@@@@@@@ 15 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=25

seed 9351 Mean_R 5509.8
#################################################################################################### 
 {'seed': 9351, 'R100': 5509.8, 'numep': 0} 
 ####################################################################################################
sum 54844.9 AVG: 5484.49

 @@@@@@@@@@ 16 @@@@@@@@@@ 

#######use epoch:  75
[17, 256, 256, 6]
##### Actor SNN

PoPSpike:
 PopSpikeEncoderRegularSpike()
##### SNN_o/p_spikeactivity
 SpikeMLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=170, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
  )
  (out_pop_layer): Linear(in_features=256, out_features=60, bias=True)
)
##### Actor_mean (mu)
 PopSpikeDecoder(
  (decoder): Conv1d(6, 6, kernel_size=(10,), stride=(1,), groups=6)
  (output_activation): Identity()
)
##### Actor_std (log_sigma)
 Sequential(
  (0): Linear(in_features=17, out_features=256, bias=True)
  (1): SELU()
  (2): Linear(in_features=256, out_features=25

seed 9351 Mean_R 5476.7
#################################################################################################### 
 {'seed': 6557, 'R100': 5510.9, 'numep': 0} 
 ####################################################################################################
sum 54885.1 AVG: 5488.51
